# **데이터 전처리**

## **데이터 로드**


In [ ]:
import pandas as pd

# 파일 경로
file_paths = {
    "2018": "/content/WHR_2018.csv",
    "2019": "/content/WHR_2019.csv",
    "2020": "/content/WHR_2020.csv",
    "2021": "/content/WHR_2021.csv",
    "2022": "/content/WHR_2022.csv",
}

# 데이터 로드
data = {year: pd.read_csv(path) for year, path in file_paths.items()}

## **결측치**

#### **1. 결측치 개수 확인**
- 데이터셋을 분석한 결과, 총 1개의 결측치가 존재
- 결측치가 포함된 변수와 개수는 다음과 같다 : <br>
perceptions_of_corruption, 1

#### **2. 결측치 처리**
- 단 1개의 결측치만 존재하여 전체 데이터 분석에 미치는 영향이 미미함.
- 별다른 왜곡 없이 분석을 진행할 수 있다고 판단.
- 따라서 추가적인 결측치 처리는 수행하지 않고 그대로 유지하기로 결정.

In [ ]:
# 각 연도별 결측치 개수 확인
missing_values = {year: df.isnull().sum() for year, df in data.items()}

# 결과 출력
for year, missing in missing_values.items():
    print(f"{year}년 데이터 결측치 개수:")
    print(missing[missing > 0]) # 결측치가 있는 컬럼만 출력

2018년 데이터 결측치 개수:
perceptions_of_corruption    1
dtype: int64
2019년 데이터 결측치 개수:
Series([], dtype: int64)
2020년 데이터 결측치 개수:
Series([], dtype: int64)
2021년 데이터 결측치 개수:
Series([], dtype: int64)
2022년 데이터 결측치 개수:
Series([], dtype: int64)


## **2020년을 기준으로 ['country'] 컬럼 전처리를 진행**


In [ ]:
# 국가명 컬럼 확인 (각 연도별)
columns_info = {year: df.columns for year, df in data.items()}

# 2022년 데이터의 컬럼명 변경
data["2022"].rename(columns={"Country": "country", "Region": "region"}, inplace=True)

# 2020년 기준 국가 목록 가져오기
country_2020 = set(data["2020"]["country"])

# 각 연도의 국가명 비교
country_mismatches = {}
for year, df in data.items():
    if year == "2020":
        continue  # 기준 데이터이므로 비교하지 않음
    mismatched_countries = set(df["country"]) - country_2020
    country_mismatches[year] = mismatched_countries

# 국가명이 일치하지 않는 경우 확인
country_mismatches

{'2018': {'Angola', 'Belize', 'Bhutan', 'Qatar', 'Somalia', 'Sudan', 'Syria'},
 '2019': {'Bhutan', 'Qatar', 'Somalia', 'Syria'},
 '2021': set(),
 '2022': {'Congo', 'Czechia', 'Eswatini'}}

## **국가 데이터 삭제 및 국가명 일치**

#### **1. 1개년에만 존재하는 국가의 경우 비교가 불가능하다고 판단하여 삭제**
- {"Angola", "Belize", "Sudan"} -> 2018년 데이터만 존재
- {"Czechia"} -> 2022년 데이터만 존재

#### **2. 2020년 데이터를 기준으로 국가명을 일치시킨다.**
- "Congo"의 경우 : <br>
2018~2021년 데이터에서는 두 개의 국가(Congo (Brazzaville), Congo (Kinshasa))로 구분되어 있으나, 2022년 데이터에서는 하나로 통합되어 나타남. <br>
따라서, 공식 사이트의 리포트를 확인한 후 Congo (Brazzaville)로 변경하는 전처리를 진행.

In [ ]:
# 삭제할 국가 (1개년 데이터에만 존재하는 국가)
countries_to_remove = {"Angola", "Belize", "Czechia", "Sudan"}

# 변경할 국가명 매핑 (2020년 기준으로 일치)
country_mapping = {
    "Congo": "Congo (Brazzaville)",
    "Eswatini": "Swaziland"
}

# 매핑 적용 및 삭제 처리
for year, df in data.items():
    # 국가명 변경
    df["country"] = df["country"].replace(country_mapping)
    # 삭제할 국가 제거
    df = df[~df["country"].isin(countries_to_remove)]
    # 수정된 데이터 반영
    data[year] = df

# 수정된 데이터를 다시 저장
updated_file_paths = {}
for year, df in data.items():
    save_path = f"/content/Updated_WHR_{year}.csv"
    df.to_csv(save_path, index=False)
    updated_file_paths[year] = save_path

updated_file_paths

{'2018': '/content/Updated_WHR_2018.csv',
 '2019': '/content/Updated_WHR_2019.csv',
 '2020': '/content/Updated_WHR_2020.csv',
 '2021': '/content/Updated_WHR_2021.csv',
 '2022': '/content/Updated_WHR_2022.csv'}

In [ ]:
# 연도별 데이터를 하나의 데이터셋으로 통합
merged_data = pd.concat(data.values(), ignore_index=True)

# 통합된 데이터 저장 경로
merged_file_path = "/content/WHR_2018_2022.csv"
merged_data.to_csv(merged_file_path, index=False)

# 저장된 파일 경로 반환
merged_file_path

'/content/WHR_2018_2022.csv'